# Testing for Language Model - Dungeon Master (LMDM)

In [1]:
# copy of imports from the original mapwalker.py file:
import json
from mapwalker_data import World, Node, Edge, Player
from typing import List, Tuple, Union, Type, Dict
import random
import sys
from pydantic import BaseModel, Field
from openai import OpenAI
import textwrap
from enum import Enum

In [2]:
#able to print big BaseModel's nicely
from devtools import pprint

In [3]:
# The URL where the local server is running
url = "http://localhost:1234/v1/"
MODEL = "qwen2.5-14b-instruct"

In [4]:
#get the client:
client = OpenAI(base_url=url, api_key="lm-studio")

In [5]:
#see some json defns:
class MovementOption(BaseModel):
    identifier: str = Field("move", literal=True)
    location: str

class InteractOption(BaseModel):
    identifier:str = Field("interact", literal=True) 
    result: str

class OutputChoice(BaseModel):
    value: Union[MovementOption, InteractOption, None]

In [6]:
#new update to lmstudio has stricter reqs for structured_output:
def update_schema(input_model: Type[BaseModel]) -> Dict[str, any]:
    tmp = input_model.model_json_schema()

    return {
        "type": "json_schema",
        "json_schema": {
            "name": "test_schema",
            "strict": True,
            "schema": tmp
        }
    }
    tmp['strict'] = True
    tmp = {"type": "json_schema", "json_schema": {"strict":True, "name": tmp['title'], "schema":tmp}}

    return tmp

def StructuredApiCall(messages: List, model_format: Type[BaseModel]):
    #first get the schema for the model:
    schema = update_schema(model_format)

    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=messages,
        response_format=update_schema(model_format),
        temperature=0.5,
    ).choices[0].message.content

    return model_format.model_validate_json(response)

In [7]:
pprint(update_schema(MovementOption))

{
    'type': 'json_schema',
    'json_schema': {
        'name': 'test_schema',
        'strict': True,
        'schema': {
            'properties': {
                'identifier': {
                    'default': 'move',
                    'literal': True,
                    'title': 'Identifier',
                    'type': 'string',
                },
                'location': {
                    'title': 'Location',
                    'type': 'string',
                },
            },
            'required': [
                'location',
            ],
            'title': 'MovementOption',
            'type': 'object',
        },
    },
}


In [38]:
pprint(update_schema(OutputChoice))

{
    'type': 'json_schema',
    'json_schema': {
        'name': 'test_schema',
        'strict': True,
        'schema': {
            '$defs': {
                'InteractOption': {
                    'properties': {
                        'identifier': {
                            'default': 'interact',
                            'literal': True,
                            'title': 'Identifier',
                            'type': 'string',
                        },
                        'result': {
                            'title': 'Result',
                            'type': 'string',
                        },
                    },
                    'required': ['result'],
                    'title': 'InteractOption',
                    'type': 'object',
                },
                'MovementOption': {
                    'properties': {
                        'identifier': {
                            'default': 'move',
                            'liter

In [12]:
#now lets ask for a result:
PARSE_CHOICE_PROMPT = textwrap.dedent(
    """
    The player is in the current node. \
    They may either interact in the node, \
    or they may attempt to move to another node. \
    Based on their prompt, decide if they are trying \
    to interact, move, or neither. \
    If they interact, describe what happens as 'result'. \
    If they move, say the name of the new node as 'location'. \
    If neither, then return None.
    """
)

In [41]:
#setup the messages:
messages = [
    {
        "role": "system",
        "content": PARSE_CHOICE_PROMPT
    },
    {
        "role": "system",
        "content": "Current node: a dark forest covered by moonlight"
    },
    {
        "role": "user",
        "content": "move to the nearby well."
    }
]


In [44]:
response = client.beta.chat.completions.parse(
    model=MODEL,
    messages=messages,
    response_format=update_schema(OutputChoice),
    temperature=0.5,
).choices[0].message.content

print(response)
print(OutputChoice.model_validate_json(response))

{"value": {"location": "a decrepit well"}}
value=MovementOption(identifier='move', location='a decrepit well')


### Now to test generation of arcs and scenes:

In [8]:
#classes for storylines:
class StoryArcResponse(BaseModel):
    name: str
    description: str

class ArcCollectionResponse(BaseModel):
    name: str
    all_arcs: List[StoryArcResponse]

class StoryScene(BaseModel):
    name: str
    description: str

class SceneCollection(BaseModel):
    all_scenes: List[StoryScene]

In [9]:
#prompt to generate arcs first
ARC_GENERATE_PROMPT = textwrap.dedent(
    """
    You are a virtual Dungeon Master, \
    crafting a solo campaign for a player. \
    Given the context given by the user, \
    Craft a rough outline of the main arcs in the story. \
    For each arc, give a name and a short description. \
    Generate prefer generating about 3 arcs. \
    Also generate a name for the campaign.
    """
)

SCENE_GENERATE_PROMPT = textwrap.dedent(
    """
    You are a virtual Dungeon Master, \
    crafting a solo campaign for a player. \
    The name of the story, as well as the \
    main arcs have already been outlined. \
    Based on the prompt (selecting one of the arcs), \
    generate an outline of the main scenes in that arc. \
    For each scene, give a name and a short description. \
    For each scene, focus mostly on where it is and what \
    npcs do. Try not to determine how the player would act. \
    Prefer generating between 3 and 6 scenes.
    """
)

In [11]:
arc_messages = [
    {
        "role": "system",
        "content": ARC_GENERATE_PROMPT 
    },
    {
        "role": "user",
        "content": ""
    }
]

In [12]:
arcs_outline = StructuredApiCall(arc_messages, ArcCollectionResponse)
pprint(arcs_outline)

ArcCollectionResponse(
    name='Shadows Over Thunderspire',
    all_arcs=[
        StoryArcResponse(
            name='The Whispering Prophecy',
            description=(
                'A mysterious prophecy begins to come true, foretelling the rise of an ancient evil. The player must u'
                'ncover the secrets of their own past and the origins of this evil, leading them on a quest through fo'
                'rgotten ruins and hidden libraries.'
            ),
        ),
        StoryArcResponse(
            name='The Marked Heir',
            description=(
                'As the player uncovers more about the prophecy, they are revealed to be the chosen heir of an ancient'
                ' lineage. This discovery brings with it new allies and enemies, as well as a series of trials that mu'
                'st be overcome to prove their worthiness.'
            ),
        ),
        StoryArcResponse(
            name="The Shadow's Gambit",
            description=(
  

In [54]:
scenes_messages = [
    {
        "role": "system",
        "content": SCENE_GENERATE_PROMPT 
    },
    {
        "role": "system",
        "content": f"Current Arcs: {arcs_outline.model_dump_json()}"
    },
    {
        "role": "user",
        "content": f"Generate Scenes for '{arcs_outline.all_arcs[0].name}'"
    }
]

In [55]:
response = client.beta.chat.completions.parse(
    messages=scenes_messages,
    model=MODEL,
    response_format=SceneCollection
)
response = response.choices[0].message.content
scenes_outline: SceneCollection= SceneCollection.model_validate_json(response)
pprint(scenes_outline)

SceneCollection(
    all_scenes=[
        StoryScene(
            name='The Village of Vespera',
            description=(
                'Our initiate begins their journey in the small, isolated village of Vespera. Here, they encounter loc'
                'al villagers who offer guidance and insight into the basics of their newfound abilities. The village '
                'is nestled on the edge of a forest, where natural elements provide both beauty and danger.'
            ),
        ),
        StoryScene(
            name='The First Stormlight',
            description=(
                'Under the guidance of an experienced elder from the Radiant Order, our initiate learns to tap into St'
                'ormlight for the first time using gemstones. This scene takes place in a secluded grove near Vespera,'
                ' under the watchful eyes of the elder who teaches them about the fundamental principles and safety pr'
                'ecautions.'
            ),
        ),
